In [1]:
import numpy as np # library for data


!conda install -c conda-forge geopy --yes # install geopy

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


! pip install geocoder # install geocoder
import json # library for JSON files

from geopy.geocoders import Nominatim   #  latitude and longitude conversion
import geocoder # coordinate library

import requests # library for requests
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup
#Kmeans clustering unsupervised
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes #map library
import folium # map rendering library

print("Libraries imported.") #check for all installed libraries

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Import data from wikipedia

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Beaches_of_Goa").text
soup = BeautifulSoup(data, 'html.parser')
goanbeaches = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    goanbeaches.append(row.text)
beaches = pd.DataFrame({"goanbeaches":goanbeaches })

beaches

,goanbeaches
0,Agonda
1,Anjuna
2,Arambol
3,"Baga, Goa"
4,Bambolim
5,Benaulim
6,Calangute
7,Candolim
8,Caranzalem
9,"Carmona, Goa"


In [3]:
def latlng(goanbeaches):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Goa, India'.format(goanbeaches))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coord = [latlng(goanbeaches) for goanbeaches in beaches["goanbeaches"].tolist() ]

In [4]:
coordinate = pd.DataFrame(coord, columns=['Latitude', 'Longitude'])

In [5]:
final = beaches.join(coordinate)
final

,goanbeaches,Latitude,Longitude
0,Agonda,15.05738,73.98343
1,Anjuna,15.58200,73.74675
2,Arambol,15.68345,73.71075
3,"Baga, Goa",15.56517,73.75517
4,Bambolim,15.45000,73.85000
5,Benaulim,15.25399,73.93354
6,Calangute,15.53700,73.76425
7,Candolim,15.51350,73.76881
8,Caranzalem,15.47400,73.81204
9,"Carmona, Goa",15.18333,73.93333


### Goa Latitude and longitude

In [6]:
address = 'Goa, India'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

15.3004543 74.0855134


In [7]:
beach_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for index, row in final.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=15,
                        popup=row['goanbeaches'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(beach_map)
beach_map

In [2]:
# define Foursquare Credentials and Version
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [9]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, goanbeaches in zip(final['Latitude'], final['Longitude'], final['goanbeaches']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            goanbeaches,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [10]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['goanbeaches', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(1176, 7)


,goanbeaches,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Agonda,15.05738,73.98343,Cola Beach,15.057386,73.971336,Beach
1,Agonda,15.05738,73.98343,Agonda White Sands,15.048538,73.984542,Hotel
2,Agonda,15.05738,73.98343,Agonda Beach,15.040155,73.988802,Beach
3,Agonda,15.05738,73.98343,Simrose resort,15.042256,73.987222,Resort
4,Agonda,15.05738,73.98343,The Bay Agonda,15.053906,73.981984,Resort
5,Agonda,15.05738,73.98343,Madhu Bar And Restaurant,15.045434,73.985914,Restaurant
6,Agonda,15.05738,73.98343,Blue Lagoon,15.057353,73.971447,Resort
7,Agonda,15.05738,73.98343,Jardim A Mar,15.042499,73.987213,Indian Restaurant
8,Agonda,15.05738,73.98343,Fatima's Corner,15.041065,73.988927,Goan Restaurant
9,Anjuna,15.58200,73.74675,Artjuna,15.580826,73.747676,Café


In [12]:
venues_df.groupby('VenueCategory')['goanbeaches'].nunique()

VenueCategory
Airport Terminal                    2
American Restaurant                 4
Asian Restaurant                   13
Australian Restaurant               4
BBQ Joint                           8
Bakery                              7
Bar                                14
Basketball Court                    1
Beach                              23
Beach Bar                           5
Bed & Breakfast                     4
Beer Garden                         1
Bistro                              3
Boat or Ferry                       1
Bookstore                           1
Brazilian Restaurant                1
Breakfast Spot                      9
Buffet                              1
Burger Joint                        8
Bus Station                         1
Café                               18
Caribbean Restaurant                5
Casino                              2
Chinese Restaurant                  9
Clothing Store                      5
Cocktail Bar                        

In [38]:
restaurant=venues_df[venues_df['VenueCategory'].str.contains(r'Restaurant')].reset_index()
X=restaurant[["goanbeaches","VenueCategory"]]
X

,goanbeaches,VenueCategory
0,Agonda,Restaurant
1,Agonda,Indian Restaurant
2,Agonda,Goan Restaurant
3,Anjuna,Restaurant
4,Anjuna,Restaurant
5,Anjuna,French Restaurant
6,Anjuna,Restaurant
7,Anjuna,Tibetan Restaurant
8,Anjuna,Seafood Restaurant
9,Anjuna,Tapas Restaurant


In [13]:
venues_df['VenueCategory'].unique()

array(['Beach', 'Hotel', 'Resort', 'Restaurant', 'Indian Restaurant',
       'Goan Restaurant', 'Café', 'Burger Joint', 'French Restaurant',
       'Breakfast Spot', 'Tibetan Restaurant', 'Seafood Restaurant',
       'Food & Drink Shop', 'Grocery Store', 'Tapas Restaurant',
       'Nightclub', 'Ice Cream Shop', 'Music Venue', 'Juice Bar',
       'Italian Restaurant', 'Diner', 'Greek Restaurant', 'Bar',
       'Asian Restaurant', 'Flea Market', 'Snack Place', 'BBQ Joint',
       'Hostel', 'Fast Food Restaurant', 'Modern European Restaurant',
       'Japanese Restaurant', 'Pub', 'Lounge', 'Pizza Place',
       'Australian Restaurant', 'Bed & Breakfast',
       'Portuguese Restaurant', 'Racetrack', 'Convenience Store',
       'Eastern European Restaurant', 'Sculpture Garden', 'Theater',
       'Public Art', 'Vegetarian / Vegan Restaurant', 'Beach Bar',
       'Gastropub', 'Mediterranean Restaurant', 'Falafel Restaurant',
       'Indian Chinese Restaurant', 'Russian Restaurant', 'Tea Room'

## consider goan beaches and venues

In [39]:
X

,goanbeaches,VenueCategory
0,Agonda,Restaurant
1,Agonda,Indian Restaurant
2,Agonda,Goan Restaurant
3,Anjuna,Restaurant
4,Anjuna,Restaurant
5,Anjuna,French Restaurant
6,Anjuna,Restaurant
7,Anjuna,Tibetan Restaurant
8,Anjuna,Seafood Restaurant
9,Anjuna,Tapas Restaurant


In [76]:
Final = pd.get_dummies(X['VenueCategory'])
Y=X[['goanbeaches']].join(Final)
kl_grouped = Y.groupby(["goanbeaches"])
kl_grouped.head()



,goanbeaches,American Restaurant,Asian Restaurant,Australian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Goan Restaurant,Greek Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Multicuisine Indian Restaurant,Portuguese Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant
0,Agonda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Agonda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Agonda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Anjuna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Anjuna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,Anjuna,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Anjuna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,Anjuna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
36,Arambol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,Arambol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

clustering = KMeans(n_clusters=5, random_state=0).fit(Final)
clustering.labels_[:]

array([2, 1, 0, 2, 2, 0, 2, 0, 3, 0, 1, 0, 4, 2, 0, 1, 0, 2, 2, 0, 0, 0,
       1, 4, 1, 2, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 2, 1, 2,
       0, 1, 4, 0, 0, 0, 1, 1, 0, 2, 2, 2, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 3,
       0, 1, 1, 0, 3, 0, 0, 0, 2, 1, 1, 0, 3, 3, 1, 1, 3, 0, 2, 2, 1, 3,
       1, 1, 2, 1, 4, 3, 1, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 3, 2, 0, 1, 0,
       0, 3, 2, 3, 3, 0, 3, 2, 1, 1, 1, 0, 1, 0, 0, 2, 2, 3, 3, 1, 3, 0,
       0, 2, 0, 2, 3, 1, 0, 2, 0, 0, 1, 0, 3, 3, 1, 0, 2, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 3, 3, 4, 0, 1, 3, 1, 2, 0, 3, 0, 2, 0, 4, 1,
       1, 0, 1, 1, 3, 0, 1, 2, 0, 2, 2, 0, 0, 2, 1, 1, 1, 3, 0, 0, 1, 3,
       3, 2, 4, 2, 3, 2, 1, 0, 0, 3, 2, 4, 0, 2, 1, 2, 1, 3, 1, 0, 0, 2,
       2, 2, 2, 3, 3, 3, 3, 0, 1, 2, 2, 3, 3, 1, 3, 3, 1, 0, 0, 4, 0, 2,
       2, 0, 0, 0, 4, 0, 0, 2, 2, 0, 4, 2, 1, 1, 1, 1, 0, 0, 0, 2, 1, 3,
       1, 0, 1, 2, 1, 1, 1, 1, 3, 0, 0, 1, 2, 3, 3,